In [1]:
import numpy as np
import math

In [15]:
def dcg_of_item(actual_location, desired_location, length):
    distance = np.abs(actual_location - desired_location)
    score = length - distance
    # the closer you are the higher the score
    denominator = np.log2(actual_location+1)
    return score/denominator
    
    

In [64]:
def calculate_score_of_ranking(desired_ranking, actual_ranking):
    score_sum = 0
    for i in range(len(desired_ranking)):
        item = actual_ranking[i] # this is the item that was in the ith place
        desired_rank_of_item = desired_ranking.index(item)+1
        score = dcg_of_item(i+1, desired_rank_of_item, len(desired_ranking))
        score_sum += score
    return score_sum

In [19]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import seaborn as sns
from ExplainableMaximumLikelihoodCalculator import ExplainableMaximumLikelihoodCalculator
import pysam
from Bio import SeqIO, Seq, SeqRecord, pairwise2
from Bio.pairwise2 import format_alignment
from LoadDataUtils import getListOfReadsFromBamFile, getListOfReadsFromFastaFile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import shap
from scipy.stats import chisquare
from scipy.special import rel_entr
fileNameSapiens =  "data/simulatedData/human_chinese_AF346973_500samples.fas" #File that contains 500 reads from homo sapiens
fileNameNeanderthals =  "data/simulatedData/Neanderthal_Goyet_KX198085_500samples.fas" #File that contains 500 reads from neanderthal
fileNameDenisovans =  "data/simulatedData/denisova_kx663333_500samples.fas" #File that contains 500 reads from denisovan
neanderthals_500_generated = getListOfReadsFromFastaFile(fileNameNeanderthals)
sapiens_500_generated = getListOfReadsFromFastaFile(fileNameSapiens)
denisovan_500_samples = getListOfReadsFromFastaFile(fileNameDenisovans)
path_to_frequencies_table = "data/substitution_matrix.tsv"

2023-07-18 19:28:08.920299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 19:28:09.767008: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-18 19:28:09.880671: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-18 19:28:11.282050: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [20]:
sapiens_reference_file_names = [
                    "data/reference_files/human_AF346981_French.fa",
                     "data/reference_files/human_AY195760_Korea.fa",
                      "data/reference_files/human_AY882416_Ethiopia.fa",
                      "data/reference_files/human_AY963586_Italian.fa",
                      "data/reference_files/human_AY195781_Caucasian.fa",
                      "data/reference_files/human_AY195757_Iraqi-Israeli.fa",
                      "data/reference_files/human_AY195749_NativeAmerican.fa"]
neanderthals_reference_file_names = [
                            "data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa",
                           "data/reference_files/Neanderthal_Altai_KC879692.fa",
                           "data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa",
                           "data/reference_files/Neanderthal_Spy_94a_MG025538.fa",
                            "data/reference_files/Neanderthal_Vindija33.16_AM948965.fa",
                            "data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa"]
denisovan_reference_file_names = [  
                        "data/reference_files/Denisova_MT576653.1.fa",
                        "data/reference_files/Denisova_MT576652.1.fa",
                        "data/reference_files/Denisova_4_FR695060.fa",
                        "data/reference_files/Denisova_8_KT780370.fa",
                        "data/reference_files/Denisova_manual_phalanx_NC_013993.fa",
                        "data/reference_files/Denisova_MT576651.1.fa"]

In [21]:
def generate_sample(sap, nean, den):
    list_before =  list(np.random.choice(sapiens_500_generated, sap)) + list(np.random.choice(neanderthals_500_generated, nean)) + list(np.random.choice(denisovan_500_samples, den))
    l = []
    for i in list_before:
        l.append(str(i))
    return l

In [22]:
from itertools import combinations

def find_all_subsets(group):
    all_subsets = []
    n = len(group)
    
    for r in range(n + 1):
        subsets_r = combinations(group, r)
        all_subsets.extend(list(subsets_r))
        
    return all_subsets

In [23]:
def calculate_shapley_values_for_datum(data_point, length_of_data, ml_calculator):
    indexes_without_datum = [i for i in range(length_of_data) if i != data_point]
    all_subsets = find_all_subsets(indexes_without_datum)
    sum_over_all_subsets = 0 
    for G in all_subsets:
        g_with = [i for i in G]
        g_with.append(data_point)
        with_datum = ml_calculator.calc_maximum_likelihood_on_subset(subset_of_indexes=g_with).values[0]
        without_datum = ml_calculator.calc_maximum_likelihood_on_subset(subset_of_indexes=G).values[0]
        with_minus_without = with_datum - without_datum
        multiplier = np.math.factorial(len(G)) * np.math.factorial(length_of_data - len(G) - 1)
        value_for_sum = with_minus_without * multiplier 
        sum_over_all_subsets += value_for_sum
    shapleys = sum_over_all_subsets / np.math.factorial(length_of_data)
    return  shapleys

In [139]:
def calculateRankingResultsForLabel(label, ground_truth_shapleys, shaps, a_s_d, shapley_values_monte_carlo):
    ranking_results = []
    ground_truth_shapley_label = [i[label] for i in ground_truth_shapleys]
    ground_shapley_label_with_index = []
    for i in range(len(ground_truth_shapley_label)):
        ground_shapley_label_with_index.append((ground_truth_shapley_label[i], i))
    ground_shapley_label_with_index_sorted = sorted(ground_shapley_label_with_index, key=lambda a:a[0], reverse=True)
    only_indexes_sorted = [i[1] for i in ground_shapley_label_with_index_sorted]

    shaps_label = shaps[label][0]
    shaps_with_index = []
    for i in range(len(shaps_label)):
        shaps_with_index.append((shaps_label[i], i))
    shap_label_with_index_sorted = sorted(shaps_with_index, key=lambda a:a[0], reverse=True)
    only_indexes_sorted_shap = [i[1] for i in shap_label_with_index_sorted]
    ranking_result_shap = calculate_score_of_ranking(only_indexes_sorted, only_indexes_sorted_shap)

    a_s_d_label = [i[label] for i in a_s_d]
    a_s_d_index = []
    for i in range(len(a_s_d_label)):
        a_s_d_index.append((a_s_d_label[i], i))
    a_s_d_with_index_sorted = sorted(a_s_d_index, key=lambda a:a[0], reverse=True)
    only_indexes_sorted_a_s_d = [i[1] for i in a_s_d_with_index_sorted]
    ranking_result_a_s_d = calculate_score_of_ranking(only_indexes_sorted, only_indexes_sorted_a_s_d)

    monte_carlo_label = shapley_values_monte_carlo[label][0]
    monte_carlo_label_with_index = []
    for i in range(len(monte_carlo_label)):
        monte_carlo_label_with_index.append((monte_carlo_label[i], i))
    monte_carlo_label_with_index_sorted = sorted(monte_carlo_label_with_index, key=lambda a:a[0], reverse=True)
    monte_carlo_only_indexes_sorted = [i[1] for i in monte_carlo_label_with_index_sorted]
    ranking_result_monte_carlo = calculate_score_of_ranking(only_indexes_sorted, monte_carlo_only_indexes_sorted)

    ranking_results.append((ranking_result_shap, ranking_result_a_s_d, ranking_result_monte_carlo))
    return ranking_results


In [153]:
def calculateRankingForAllALgorithms():
    data = generate_sample(5,2,0)
    maximum_likelihood_calculator_d_1 = ExplainableMaximumLikelihoodCalculator(data,
                                                        ref_neanderthal_file_names=neanderthals_reference_file_names,
                                                        ref_sapien_file_names=sapiens_reference_file_names,
                                                        ref_denisovan_file_names=denisovan_reference_file_names,
                                                        path_to_substitution_matrix=path_to_frequencies_table,
                                                        number_of_jobs=-1)
    ground_truth_shapleys = []
    for i in range(len(data)):
        ground_truth_shapleys.append(calculate_shapley_values_for_datum(i,len(data),maximum_likelihood_calculator_d_1))
    shaps = maximum_likelihood_calculator_d_1.calculate_shapley_values()
    a_s_d = maximum_likelihood_calculator_d_1.get_A_s_d_values()
    shapley_values_monte_carlo = maximum_likelihood_calculator_d_1.estimate_shapley_values(number_of_samples_per_read = 200)[1]

    results = []
    for i in range(3):
        result = calculateRankingResultsForLabel(i, ground_truth_shapleys, shaps, a_s_d, shapley_values_monte_carlo)
        results.append(result)

    results_shap = np.mean([i[0][0] for i in results])
    results_a_s_d = np.mean([i[0][1] for i in results])
    results_monte_carlo = np.mean([i[0][2] for i in results])


    return((results_shap, results_a_s_d, results_monte_carlo))



In [154]:
all_scores = []
for i in range(3):
    res = calculateRankingForAllALgorithms()
    all_scores.append(res)
    print(res)

Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  7
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
126
Start working on read number 0 in processId 2521
Start working on read number 1 in processId 2522
Start working on read number 2 in processId 2523
Start working on read number 3 in processId 2524Start working on read number 4 in processId 2525

Start working on read number 5 in processId 2526
Start working on read number 6 in processId 2527
(23.576683015630323, 22.60384455584519, 21.042519964370715)
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  7
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
126
Start working on read number 0 in processId 2608
Start working on read number 2 in processId 2610Start working on read number 1 in processId 2609

Start working on read number 3 in processId 2611Start working on read number 5 in processId 2613Start working on read number 4 in processId 2612

Start working on read number 6 in processId 2614

(23.245046779953224, 21.427148882700994, 24.068485453506465)
Loading sequences and calculating alignments to all references, this might take a while. Number of reads:  7
start working on read number 0
1


  0%|          | 0/1 [00:00<?, ?it/s]

1
126
Start working on read number 0 in processId 2695
Start working on read number 2 in processId 2697Start working on read number 1 in processId 2696Start working on read number 3 in processId 2698


Start working on read number 4 in processId 2699
Start working on read number 5 in processId 2700
Start working on read number 6 in processId 2701
(24.05357520165479, 25.089020890721415, 24.222598496904553)


In [155]:
all_scores

[(23.576683015630323, 22.60384455584519, 21.042519964370715),
 (23.245046779953224, 21.427148882700994, 24.068485453506465),
 (24.05357520165479, 25.089020890721415, 24.222598496904553)]

In [169]:
results_shap = np.mean([i[0] for i in all_scores])
results_a_s_d = np.mean([i[1] for i in all_scores])
results_monte_carlo = np.mean([i[2] for i in all_scores])